In [1]:
import torch
torch.cuda.is_available(), torch.cuda.is_bf16_supported()

(True, True)

In [2]:
DEVICE = "cuda" if torch.cuda.is_available else "cpu"

In [3]:
seq_len = 512
head_size = 32

In [4]:
q = torch.randn(seq_len, head_size).to(DEVICE)
k = torch.randn(seq_len, head_size).to(DEVICE)
v = torch.randn(seq_len, head_size).to(DEVICE)

In [15]:
from math import sqrt
M = 1024
# Block col size
B_c = M//(4 * head_size)
B_r = min(M//(4 * head_size), head_size)

O = torch.zeros(seq_len, head_size).to(DEVICE)
l = torch.zeros(seq_len).to(DEVICE)
m = torch.fill(torch.zeros(seq_len), float('-inf')).to(DEVICE)
# mask = torch.tril(torch.ones(B_r, B_c)).to(DEVICE)
parameters = [q, k, v]
for i in parameters:
    i.requires_grad_(True)

# iterate over seqlen/B_c blocks of K and V
T_c = seq_len // B_c
T_r = seq_len // B_r
for j in range(T_c):

    # (B_c x head_size)
    Kj = k[B_c * j:B_c * (j+1), :]
    Vj = v[B_c * j:B_c * (j+1), :]

    k_idx = torch.arange(B_c * j, B_c * (j + 1)).unsqueeze(0)

    for i in range(T_r):
        if i < j:
            continue

        # (B_r x head_size)
        Qi = q[B_r * i:B_r * (i+1), :]
        scale = 1.0 / sqrt(head_size)

        # (B_r x head_size) @ (head_size x B_c)
        # (B_r x B_c)
        Sij = (Qi @ Kj.T) * scale

        q_idx = torch.arange(B_r * i, B_r * (i + 1)).unsqueeze(1)
        # B_r x B_c
        mask = (q_idx >= k_idx).to(DEVICE)

        # THE FUCKING BUG WAS HERE BUG BUG BUG FUCKKKKK
        # here's a tip, heer. masked_fill is not in-place. masked_fill_ is
        
        Sij.masked_fill_(~mask, float('-inf'))
        # (B_r)
        mi_old = m[B_r * i : B_r * (i + 1)]

        # (B_r)
        li_old = l[B_r * i : B_r * (i + 1)]

        # (B_r x head_size)
        oi_old = O[B_r * i : B_r * (i + 1), :]

        # torch max returns max of whole matrix if dim not specified.
        # dim=1 searches for max across rows and along columns 
        # 0th index has the max values and 1st index has the indices of max values
        # (B_r)
        mij = torch.max(Sij,dim=1)[0]
                
        # (B_r x B_c) - (B_r) would be broadcasted to (B_r x B_r) and applied row wise
        # unsqueeze(1) will make it (B_r x 1) and then it will be broadcasted column wise.
        Pij = torch.exp(Sij - mij.unsqueeze(1))
        lij = torch.sum(Pij, dim=1)

        # stacked shape (2, B_r)
        # max needs to be (B_r)
        # reduce axis 0?
        mi_new = torch.maximum(mi_old, mij)
        
        li_new = (torch.exp(mi_old - mi_new) * li_old + torch.exp(mij - mi_new) * lij)
        # print(Sij.shape, Vj.shape)
        # print(mi_old.shape)
        # print(oi_old.shape)
        # print(li_old.shape)
        # print(torch.diag(li_old).shape)
        # print(torch.exp(mi_old - mi_new).shape)
        old_part = (oi_old * li_old.unsqueeze(1)) * torch.exp(mi_old - mi_new).unsqueeze(1)
        # print(old_part.shape)
        new_part = Pij @ Vj * (torch.exp(mij - mi_new).unsqueeze(1))
        # print(new_part.shape)
        # print(torch.diag(li_new).inverse())
        Oi_new = ((old_part + new_part) / li_new.unsqueeze(1))
        # print(Oi_new)
        O[B_r * i : B_r * (i + 1), :] = Oi_new 
        m[B_r * i : B_r * (i + 1)] = mi_new
        l[B_r * i : B_r * (i + 1)] = li_new


In [16]:
O

tensor([[-0.8185, -0.5016,  2.4753,  ...,  0.6985,  0.3798, -0.5203],
        [-0.6160, -0.6139,  1.2790,  ...,  0.4495,  0.2233, -0.2138],
        [-0.4099, -0.9314,  0.3853,  ..., -0.1990, -0.8945,  0.0315],
        ...,
        [ 0.0433,  0.0072, -0.0419,  ...,  0.0544, -0.1109, -0.0151],
        [ 0.0207,  0.0935, -0.0997,  ...,  0.0177, -0.1006,  0.0235],
        [ 0.0280,  0.0298, -0.0399,  ...,  0.0534, -0.0121, -0.0324]],
       device='cuda:0', grad_fn=<CopySlices>)

In [17]:
attn = torch.nn.functional.scaled_dot_product_attention(q, k, v, is_causal=True)
attn

tensor([[-0.8185, -0.5016,  2.4753,  ...,  0.6985,  0.3798, -0.5203],
        [-0.6160, -0.6139,  1.2790,  ...,  0.4495,  0.2233, -0.2138],
        [-0.4099, -0.9314,  0.3853,  ..., -0.1990, -0.8945,  0.0315],
        ...,
        [ 0.0433,  0.0072, -0.0419,  ...,  0.0544, -0.1109, -0.0151],
        [ 0.0207,  0.0935, -0.0997,  ...,  0.0177, -0.1006,  0.0235],
        [ 0.0280,  0.0298, -0.0399,  ...,  0.0534, -0.0121, -0.0324]],
       device='cuda:0', grad_fn=<MmBackward0>)

In [ ]:
for i in parameters:
    i.grad = None
    O.backward()
    print(v.grad)

RuntimeError: grad can be implicitly created only for scalar outputs